# Food Spoilage Kernel Case Study

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
from matplotlib.lines import Line2D

In [2]:
# -------------------------
# 1) Define Internal Data
# -------------------------
system_specs = {
    "Serv": {"area": 2.93, "power": 17.7451},
    "Qerv": {"area": 3.68, "power": 21.0650},
    "Herv": {"area": 4.50, "power": 24.9934},
}
# In number of cycles - this is still from analytical model, not from RTL
execution_time = {
    "DT-Small": {"Serv": 4585, "Qerv": 2425, "Herv": 2065},
    "DT-Medium": {"Serv": 4690, "Qerv": 2458, "Herv": 2086},
    "DT-Large": {"Serv": 5215, "Qerv": 2719, "Herv": 2303},
    "KNN-Small": {"Serv": 2307410, "Qerv": 1198970, "Herv": 1014230},
    "KNN-Medium": {"Serv": 11404680, "Qerv": 5925360, "Herv": 5012140},
    "KNN-Large": {"Serv": 22779890, "Qerv": 11834258, "Herv": 10009986},
    "LDA": {"Serv": 30100, "Qerv": 15988, "Herv": 13636},
    "MLP": {"Serv": 787675, "Qerv": 293011, "Herv": 210567}
}

rf_sram = 128 # Implementing register file in SRAM for space-saving


# Define SRAM and LPROM constants (in bits)
sram_power = 0.0018
sram_area = 0.00176
lprom_power = 0 # negligible
lprom_area = 0.000359
sram_overhead_power = 0.0002 # per bit
sram_overhead_area = 0.0003 # per bit

# In bytes
sram = {
    "DT-Small": 57,
    "DT-Medium": 57,
    "DT-Large": 61,
    "KNN-Small": 125,
    "KNN-Medium": 125,
    "KNN-Large": 125,
    "LDA": 137,
    "MLP": 713
}

# In bytes
lprom = {
    "DT-Small": 240,
    "DT-Medium": 300,
    "DT-Large": 456,
    "KNN-Small": 7407,
    "KNN-Medium": 28719,
    "KNN-Large": 55359,
    "LDA": 1543,
    "MLP": 5891
}

# Define carbon intensity values for each power source
# gCO2 eq / kWhr
# Source: https://www.eia.gov/tools/faqs/faq.php?id=74&t=11
# And: https://www.energy.gov/eere/wind/articles/how-wind-can-help-us-breathe-easier
carbon_intensity_values = {
    "US Avg (2023)": "367",
    "Petroleum": "1116",
    "Coal": "1048",
    "Solar": "28",
    "Nuclear": "13",
    "Wind": "12",
    "Custom": ""
}

embodied_values = {
    "DT-Small": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.73},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.82},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.91},
    },
    "DT-Medium": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.75},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.83},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.93},
    },
    "DT-Large": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.81},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.89},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 0.98},
    },
    "KNN-Small": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.12},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.20},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.29},
    },
    "KNN-Medium": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 9.85},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 9.94},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 10.03},
    },
    "KNN-Large": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 18.27},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 18.35},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 18.44},
    },
    "LDA": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 1.29},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 1.37},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 1.46},
    },
    "MLP": {
        "Serv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.71},
        "Qerv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.79},
        "Herv": {"Carbon Footprint Min (gCO2e)": 0, "Carbon Footprint Max (gCO2e)": 3.88},
    }
}
    

In [3]:
def compute_total_carbon(
    system: str,
    workload: str,
    lifetime_yrs: float,
    inf_freq: float, # per year
    carbon_intensity: float,
    
    core_freq: float  # New input for core frequency
):
    """
    Compute the total carbon footprint of a given system with the user-specified parameters.
    Returns (embodied_carbon, operational_carbon, total_carbon).
    """
    
    sram_bytes = sram[workload] + rf_sram
    lprom_bytes = lprom[workload]

    # 1) Calculate total area
    core_area = system_specs[system]["area"]
    device_area = core_area + (sram_area + sram_overhead_area) * sram_bytes * 8 + lprom_area * lprom_bytes * 8
    # 2) Embodied carbon (simplified)

    embodied_carbon = embodied_values[workload][system]["Carbon Footprint Max (gCO2e)"]
    # 3) Operational carbon
    #    - power_of_system + power_of_sram + power_of_lprom
    #    - execution_time * inferences
    #    - carbon intensity factor
    system_power = system_specs[system]["power"]
    total_power = system_power + (sram_power + sram_overhead_power) * (sram_bytes + 128) * 8 + lprom_power * lprom_bytes * 8
    
    # Convert lifetime in years to total inferences:
    # For example: inferences_per_day * 365 * lifetime_years
    total_inferences = inf_freq * lifetime_yrs    # Execution time of each inference
    time_per_inference = execution_time[workload][system] / core_freq  # Adjusted with core frequency
    total_energy = total_power * time_per_inference * total_inferences  # (Power * Time)
    operational_carbon = total_energy * carbon_intensity
    
    total_carbon = embodied_carbon + operational_carbon

    return embodied_carbon, operational_carbon, total_carbon


In [ ]:
workload_accuracies = {
    "DT-Small": 414 / 555,
    "DT-Medium": 465 / 555,
    "DT-Large": 528 / 555,
    "KNN-Large": 549 / 555,
    "KNN-Medium": 547 / 555,
    "KNN-Small": 535 / 555,
    "LDA": 545 / 555,
    "MLP": 507 / 555
}

def plot_accuracy_vs_carbon(lifetime_years, tasks_per_year, carbon_intensity):
    x_acc = []
    y_cf = []
    labels = []
    colors = []
    carbon_intensity = float(carbon_intensity) / 3600 / 1000000
    variant_color_map = {
        'Serv': 'tab:blue',
        'Qerv': 'tab:orange',
        'Herv': 'tab:green'
    }
    x_acc = []
    y_cf = []
    labels = []
    colors = []
    markers = []
    sizes = []

    STAR_SIZE = 100
    X_SIZE = 30

    for workload, acc in workload_accuracies.items():
        footprints = []
        for variant in ['Serv', 'Qerv', 'Herv']:
            footprint = compute_total_carbon(variant, workload, lifetime_years, tasks_per_year, carbon_intensity, 10000)[2]
            footprints.append((variant, footprint))
        # Find the variant with the smallest footprint
        min_footprint = min(footprints, key=lambda x: x[1])[1]
        footprints.sort(key=lambda x: x[1], reverse=True)
        for variant, footprint in footprints:
            x_acc.append(acc)
            y_cf.append(footprint)
            colors.append(variant_color_map[variant])
            if footprint == min_footprint:
                labels.append(f"{workload}")
            else:
                labels.append("")

            if footprint != min_footprint or workload in ["MLP", "KNN-Small"]:
                markers.append('x')
                sizes.append(X_SIZE)
            else:
                markers.append('*')
                sizes.append(STAR_SIZE)

    plt.figure(figsize=(10, 6))
    plt.yscale('log')
    # Plot each point with its marker and size
    for x, y, c, m, s in zip(x_acc, y_cf, colors, markers, sizes):
        plt.scatter(x, y, c=c, s=s, marker=m, alpha=0.9)

    # Place labels to the bottom right of the datapoint
    for i, label in enumerate(labels):
        if label:  # Only annotate non-empty labels
            # xytext=(+8, -8): right and down from the point
            plt.annotate(
                label,
                (x_acc[i], y_cf[i]),
                textcoords="offset points",
                xytext=(0, -8),
                ha='center',
                va='top',
                fontsize=9
            )

    # Custom legend for the variants and marker types
    legend_elements = [
        Line2D([0], [0], marker='*', color='w', label='Pareto-Optimal', markerfacecolor='gray', markeredgecolor='k', markersize=14),
        Line2D([0], [0], marker='x', color='w', label='Not Pareto-Optimal', markerfacecolor='gray', markeredgecolor='k', markersize=8),
        Line2D([0], [0], marker='o', color='w', label='Serv', markerfacecolor='tab:blue', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Qerv', markerfacecolor='tab:orange', markersize=10),
        Line2D([0], [0], marker='o', color='w', label='Herv', markerfacecolor='tab:green', markersize=10)
    ]
    plt.legend(handles=legend_elements, title="Variant / Marker")

    plt.xlabel("Accuracy")
    plt.ylabel(f"Total Carbon Footprint over {lifetime_years:.1f} years (gCO2e)")
    plt.title("Accuracy vs. Carbon Footprint")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Reasonable slider ranges for interactive exploration
interact(
    plot_accuracy_vs_carbon,
    lifetime_years=FloatSlider(value=1, min=0.1, max=10, step=0.1, description='Lifetime (years)'),
    tasks_per_year=IntSlider(value=365, min=1, max=10000, step=1, description='Tasks/year'),
    carbon_intensity=Dropdown(
        options=carbon_intensity_values,
        value=carbon_intensity_values["US Avg (2023)"],
        description='Carbon Intensity\n(gCO2e/Wh)'
    )
)


interactive(children=(FloatSlider(value=1.0, description='Lifetime (years)', max=10.0, min=0.1), IntSlider(val…

<function __main__.plot_accuracy_vs_carbon(lifetime_years, tasks_per_year, carbon_intensity)>